In [1]:
import datetime as dt
import pandas as pd
import os
import cisco_fiscal_calendar as cfc
import compass_readouts as cr

C:\Users\phsheari\Notebooks\compass_utils\compass_readouts.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step_one['Readout_Date'] = step_one['Readout_Date'].apply(lambda x: pd.to_datetime(x))
C:\Users\phsheari\Notebooks\compass_utils\compass_readouts.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guidf['GUID'] = guidf['GUID'].apply(stringify_element)


In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
weeks = 9, 10, None

In [4]:
nonecount = 0

In [5]:
for item in weeks:
    if item == None:
        nonecount +=1
    
weeks_length = len(weeks) - nonecount

In [6]:
weeks_length

2

In [7]:
weekA = weeks[0]
weekB = weeks[1]
weekC = weeks[2]

In [8]:
if weekC == None:
    endfilename = f'FINBI_EMEAR_Bookings_FY22Q3_W{weekA}_W{weekB}.xlsx'
else:
    endfilename = f'FINBI_EMEAR_Bookings_FY22Q3_W{weekA}_W{weekB}_W{weekC}.xlsx'
print(endfilename)

FINBI_EMEAR_Bookings_FY22Q3_W9_W10.xlsx


In [9]:
today = dt.datetime.today() #.isoformat()[:10]

In [10]:
weekday_dict = {0:1,       # 0 = Monday, 1 = Tuesday, 2 = Wednesday, 3 = Thursday
                1:2,
                2:3,
                3:4,
                4:5}

In [11]:
todays_weekday = today.weekday()

In [12]:
todays_weekday

3

In [13]:
if weekC == None:
    weeks_ago = today - dt.timedelta((weeks_length * 7)+(weekday_dict[todays_weekday]))      #Looking two weeks back + 2 days. This is normall run on a Monday, so we want that prior Saturday, 2 weeks ago
    weeks_ago = weeks_ago.isoformat()[:10]
else:
    weeks_ago = today - dt.timedelta((weeks_length * 7)+(weekday_dict[todays_weekday]))      #Looking three weeks back + 2 days. This is normall run on a Monday, so we want that prior Saturday, 3 weeks ago
    weeks_ago = weeks_ago.isoformat()[:10]

In [14]:
print(weeks_ago)

2022-03-27


In [15]:
cal = cfc.get_calendar_dataframe()

In [16]:
today.isoformat()[:10]

'2022-04-14'

In [17]:
cal.loc[cal['date']==today.isoformat()[:10], 'finbi_fiscal_week_id'].values[0]

2022093

In [18]:
start_fiscal_week = cal.loc[cal['date']==weeks_ago, 'finbi_fiscal_week_id'].values[0]
end_fiscal_week = cal.loc[cal['date']==today.isoformat()[:10], 'finbi_fiscal_week_id'].values[0]

In [19]:
print(start_fiscal_week, end_fiscal_week)

2022091 2022093


In [20]:
def get_reporting_week(finbi_fiscal_week_id):
    return cal.loc[cal['finbi_fiscal_week_id']==finbi_fiscal_week_id, 'week_of_fiscalquarter'].values[0]

In [21]:
def convert_nan_to_int(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return int(dataframe_cell)
            except:
                return pd.NA

In [22]:
def convert_nan_to_int_to_str(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return str(int(dataframe_cell))
            except:
                return pd.NA

In [23]:
def drop_columns(df, column_list):
    for col in df.columns:
        if col in column_list:
            df.drop(columns=col, inplace=True)

In [24]:
today = dt.datetime.today() #.isoformat()[:10]

In [25]:
ten_months_ago = today + dt.timedelta(weeks = -43, days = -3)

In [26]:
excel_file_path_sfdc_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'
compass_file = pd.read_pickle(excel_file_path_sfdc_version)

In [27]:
# compass_file['IndexID'] = pd.NA
# compass_file['Sheet Name'] = pd.NA

In [28]:
compass_file = compass_file[['Request ID', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','GUID', 'Customer Name', 'Deal ID', 'Campaign', 'Compass Advisor','Prehash_Entity',
       'Readout_Date', 'Readout Delivery Status', 'Items Recommended','Campaign Type', 'CONCIERGE_ADVISOR_ASSIGNED','CONCIERGE_INTERNAL_READOUT_DT', ]].drop_duplicates(keep='first')

In [29]:
compass_file['Deal ID'] = compass_file.apply(lambda x: convert_nan_to_int(x['Deal ID']), axis=1)

In [30]:
compass_file['GUID'] = compass_file.apply(lambda x: convert_nan_to_int(x['GUID']), axis=1)

In [31]:
compass_file = compass_file.loc[~compass_file['Readout Delivery Status'].isin(['Declined','Postponed']) | (compass_file['CONCIERGE_INTERNAL_READOUT_DT'].notnull())]

In [32]:
compass_file.shape

(35614, 17)

In [33]:
ro = cr.get_readout_dataframe()

In [34]:
ro

,Request ID,GUID,Readout_Date
101,117567,444444,2022-04-12
102,117612,46883,2022-04-13
103,117592,28090,2022-04-13
104,117565,147274,2022-04-13
265,117502,53395,2022-04-11
...,...,...,...
68575,118288,34615,2022-04-12
68727,118505,124163,2022-04-28
68728,118506,124163,2022-04-28
68729,118507,124163,2022-04-28


In [35]:
ro = ro.loc[ro['Readout_Date'] > ten_months_ago]

In [36]:
ro.drop_duplicates(keep="first", inplace=True)

<ipython-input-36-a212905fe4a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ro.drop_duplicates(keep="first", inplace=True)


In [37]:
ro.shape

(14522, 3)

In [38]:
ro = pd.merge(ro, compass_file, how='left', left_on='Request ID', right_on='Request ID')

In [39]:
compass_drop_columns = ['Readout_Date_x','GUID_y']

In [40]:
drop_columns(ro, compass_drop_columns)

In [41]:
ro.rename(columns={'GUID_x':'GUID', 'Customer Name':'Compass GUID Name','Deal ID':'Compass Deal ID', 'Readout_Date_y':'Readout_Date'}, inplace=True)

In [42]:
# ro.rename(columns={'Deal ID':'Compass Deal ID', 'Readout_Date_y':'Readout_Date'}, inplace=True)

In [43]:
ro.drop(ro.loc[ro['Readout_Date'].isnull()].index, inplace=True)

In [46]:
ro

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,117567,444444,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET NAMED,BLUE RIDGE COMMUNICATIONS INC,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Term,BL444444,2022-04-12,NaN,NaN,COVERAGE AT RENEWAL,NaN,NaT
1,117612,46883,APJC,ASEAN_AREA,ASEAN_SG,ASEAN_COM_SGP,XILINX INC,<NA>,TELEMETRY,Houman Asefi,XI46883,2022-04-13,Emailed Recommendations,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",TELEMETRY,NaN,NaT
2,117592,28090,APJC,GREATER_CHINA,CN MNC,CN_MNC_COMM_SL,CONCENTRIX GROUP,<NA>,TELEMETRY,Houman Asefi,CO28090,2022-04-13,Emailed Recommendations,"BCS,DNA,EC-AM,EC-HTOM,IRR,SNTC,SSPT,ST",TELEMETRY,NaN,NaT
3,117565,147274,APJC,APJ_SP,APJ_SP_JAPAN,JAPAN_SP_SOFTBANK_GSP,SOFTBANK,<NA>,TELEMETRY,Houman Asefi,SO147274,2022-04-13,Emailed Recommendations,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",TELEMETRY,NaN,NaT
7,117205,33900,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,ST FRANCIS SPECIALTY HOSPITAL,<NA>,COVERAGE PROGRAM - TELEMETRY,Gloria Canales,ST33900,2022-04-11,NaN,NaN,COVERAGE PROGRAM,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14516,118287,34615,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-CENTRALEAST OPERATION,MICHIGAN MEDICINE,44542652,REACTIVE,Emma Baird,MI34615,2022-05-02,NaN,NaN,FIELD,NaN,NaT
14518,118505,124163,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,CREDIT SUISSE SINGAPORE,<NA>,REACTIVE,Term,CR124163,2022-04-28,NaN,NaN,FIELD,NaN,NaT
14519,118506,124163,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,CREDIT SUISSE UNITED STATES,<NA>,REACTIVE,Term,CR124163,2022-04-28,NaN,NaN,FIELD,NaN,NaT
14520,118507,124163,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,CREDIT SUISSE UNITED KINGDOM,<NA>,REACTIVE,Term,CR124163,2022-04-28,NaN,NaN,FIELD,NaN,NaT


In [49]:
ro = ro.loc[ro['Sales Level 1'].str.contains('emear',na=False, case=False)]

In [50]:
sfdc_full_export_path = r'C:\Users\phsheari\Documents\SFDC Data\FULL EXPORT'
services_source = pd.read_pickle(sfdc_full_export_path + r'\COMBINED.pkl')

In [51]:
services_source.drop_duplicates(keep='first', inplace=True)

In [52]:
services_source

,Deal ID,Service Source
0,20041472,New
1,20071511,Renewal
2,19409975,Renewal
3,18838389,Renewal
4,19156888,New
...,...,...
625518,18700727,New
625519,16512267,New
625520,42875086,New
625521,19511983,Renewal


In [53]:
CONVERTERS = {'End Customer Global Ultimate ID': convert_nan_to_int, 'CAV¬†End Customer ID':convert_nan_to_int , 'Deal ID': convert_nan_to_int}

In [54]:
renamefields = {'End Customer Global Ultimate Name':'GUID Name','ERP Deal ID':'Deal ID','New/Renewal':'Service Source',
       'End Customer Global Ultimate ID':'GUID', 'CAV End Customer Name':'CAV Name',
       'CAVÂ End Customer ID':'CAV ID', 'CAV End Customer ID':'CAV ID','CA Service Bookings Net':'TCV', 'Annual Bookings Net':'ACV'}

In [55]:
fin_path = r'C:\Users\phsheari\Documents\FIN BI Data\3_EMEAR'
entries = os.listdir(fin_path)

In [56]:
BCSlist = []

for f_name in entries:
    if f_name.startswith('EMEAR_AS BCS') and f_name.endswith('.xlsx'):
        tempdf = pd.read_excel(fin_path + "\\" + f_name, converters = CONVERTERS, skiprows=3)
        BCSlist.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [57]:
bcs = pd.concat(BCSlist)
bcs['AS Offer Type']='BCS'

In [58]:
bcs.columns

Index(['Unnamed: 0', 'Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category',
       'Finance Product Family/Allctd Svcs', 'Sales Level 1', 'Sales Level 2',
       'Sales Level 3', 'Sales Level 4', 'End Customer Global Ultimate Name',
       'End Customer Global Ultimate ID', 'CAV End Customer Name',
       'CAV End Customer ID', 'Deal ID', 'New/Renewal', 'AS Offer Type',
       'Partner Name', 'Finance Sub Group/Contract Type',
       'CX Product Portfolio', 'CX Product Category', 'CX Product',
       'Product ID', 'CA Service Bookings Net', 'Annual Bookings Net'],
      dtype='object')

In [60]:
bcs.shape

(1703, 26)

In [61]:
HTEClist = []

for f_name in entries:
    if f_name.startswith('EMEAR_TS HTEC') and f_name.endswith('.xlsx'):
        tempdf = pd.read_excel(fin_path + "\\" + f_name, converters = CONVERTERS, skiprows=3)
        HTEClist.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [62]:
def get_offer_type(cx_product_category):
    if pd.isnull(cx_product_category):
        return pd.NA
    elif cx_product_category.lower() == 'expert care':
        return 'HTEC'
    elif cx_product_category.lower() == 'solution support':
        return 'Solution Support'
    else:
        return pd.NA

In [63]:
htec = pd.concat(HTEClist)
# htec['Finance Program Brand'] = 'HTEC'
htec.rename(columns={'Finance Program Brand':'AS Offer Type'}, inplace=True)

In [64]:
htec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218023 entries, 0 to 218022
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         0 non-null       float64
 1   Fiscal Year                        218023 non-null  int64  
 2   Fiscal Quarter ID                  218023 non-null  object 
 3   Fiscal Period ID                   218023 non-null  int64  
 4   Fiscal Week ID                     218023 non-null  int64  
 5   Finance BU/Service Category        218023 non-null  object 
 6   Sales Level 1                      218023 non-null  object 
 7   Sales Level 2                      218023 non-null  object 
 8   Sales Level 3                      218023 non-null  object 
 9   Sales Level 4                      218023 non-null  object 
 10  End Customer Global Ultimate Name  218023 non-null  object 
 11  End Customer Global Ultimate ID    2177

In [65]:
htec['AS Offer Type'] = htec['CX Product Category'].apply(get_offer_type)

In [66]:
Successlist = []

for f_name in entries:
    if f_name.startswith('EMEAR_Success') and f_name.endswith('.xlsx'):
        tempdf = pd.read_excel(fin_path + "\\" + f_name, converters = CONVERTERS, skiprows=3)
        Successlist.append(tempdf)

In [67]:
if Successlist:
    success = pd.concat(Successlist)
else:
    success = None

In [68]:
success

,Unnamed: 0,Fiscal Year,Fiscal Quarter ID,Fiscal Period ID,Fiscal Week ID,Finance BU/Service Category,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,...,CAV End Customer ID,Deal ID,New/Renewal,Partner Name,CX Product Portfolio,CX Product Category,CX Product,Product ID,CA Service Bookings Net,Annual Bookings Net
0,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,EMEAR-REGION,EMEAR-CENTRAL,CLUSTER_CIS,COUNTRY_UKRAINE,...,1571010,55535401,NEW,CS INTEGRA ltd,Success Tracks,Level 1,Campus Network L1,CON-CSER-OA,0.00,0.00
1,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,EMEAR-REGION,EMEAR-CENTRAL,CLUSTER_CIS,COUNTRY_UKRAINE,...,1571010,55535401,NEW,CS INTEGRA ltd,Success Tracks,Level 1,Campus Network L1,CON-CSIA-OA,0.00,0.00
2,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,EMEAR-REGION,EMEAR-CENTRAL,CLUSTER_CIS,COUNTRY_UKRAINE,...,1571010,55535401,RENEW,CS INTEGRA ltd,Success Tracks,Level 1,Campus Network L1,CON-CSER-OA,0.00,0.00
3,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,EMEAR-REGION,EMEAR-CENTRAL,CLUSTER_CIS,COUNTRY_UKRAINE,...,1571010,55535401,RENEW,CS INTEGRA ltd,Success Tracks,Level 1,Campus Network L1,CON-CSIA-OA,0.00,0.00
4,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_RUSSIA,COMMERCIAL_RUS,...,115215,43369152,NEW,NVISION GROUP,Success Tracks,Level 1,Campus Network L1,CON-CSER-OA,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,NaN,2022,2022Q3,202209,2022093,TECHNICAL SUPPORT SERVICES,EMEAR-REGION,EMEAR-SOUTH,COUNTRY_ITALY,PUBLIC_SECTOR_ITA,...,454262,55797499,UNKNOWN,LUTECH S.P.A.,Success Tracks,Level 1,Campus Network L1,CON-L1SWT-OA-TS,"6,275.36","1,670.02"
1778,NaN,2022,2022Q3,202209,2022093,TECHNICAL SUPPORT SERVICES,EMEAR-REGION,EMEAR-UKI,SMALL_EAW,SBU_EAW_LONDON_L4,...,108042,44184238,UNKNOWN,"ConvergeOne, Inc.",Success Tracks,Level 1,Campus Network L1,CON-CSAL-OA,3.79,1.26
1779,NaN,2022,2022Q3,202209,2022093,TECHNICAL SUPPORT SERVICES,EMEAR-REGION,EMEAR-UKI,SMALL_EAW,SBU_EAW_LONDON_L4,...,108042,44184238,UNKNOWN,"ConvergeOne, Inc.",Success Tracks,Level 1,Campus Network L1,CON-L1SWT-OA-TS,78.66,26.22
1780,NaN,2022,2022Q3,202209,2022093,TECHNICAL SUPPORT SERVICES,EMEAR-REGION,EMEAR-UKI,SMALL_EAW,SBU_EAW_LONDON_L4,...,108042,44184238,UNKNOWN,"ConvergeOne, Inc.",Success Tracks,Level 1,Unknown L1,CON-CSAL-OA,"2,200.03",861.69


In [69]:
if success is not None:
    success = success.loc[(success['Fiscal Week ID'] >= start_fiscal_week) & (success['Fiscal Week ID'] < end_fiscal_week)]

In [70]:
#if success is not None:
success.shape

(546, 23)

In [71]:
# success = success.loc[success['CA Service Bookings Net']>1]

In [72]:
SNTC_list = []

for f_name in entries:
    if f_name.startswith('EMEAR_SNTC') and f_name.endswith('.xlsx'): 
        tempdf = pd.read_excel(fin_path + "\\" + f_name, skiprows=3, converters = CONVERTERS)
        SNTC_list.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [73]:
sntc = pd.concat(SNTC_list)
sntc.rename(columns={'Finance Program Brand':'AS Offer Type'}, inplace=True)

In [74]:
sntc.rename(columns=renamefields, inplace=True)

In [75]:
sntc.shape

(249825, 25)

In [76]:
dropcolumns = ['Unnamed: 0','Finance Product Sub Group Description','Finance Product Family/Allctd Svcs', 'Sales Level 5',
       'Sales Level 6', 'SSS Flag', 'SPM Coverage','DE/PSSM Flag','PDE/PSSM Flag','Contract Number','Finance Sub Group/Contract Type','CX Product Portfolio','CX Product']

In [77]:
if success is not None:
    for col in success.columns:
        if col in renamefields.keys():
            success.rename(columns = {col:renamefields[col]}, inplace=True)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [78]:
if success is not None:
    drop_columns(success, dropcolumns)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [79]:
if bcs is not None:
    drop_columns(bcs, dropcolumns)

In [80]:
if htec is not None:
    drop_columns(htec, dropcolumns)

In [81]:
if bcs is not None:
    for col in bcs.columns:
        if col in renamefields.keys():
            bcs.rename(columns={col:renamefields[col]}, inplace=True)

In [82]:
if htec is not None:
    for col in htec.columns:
        if col in renamefields.keys():
            htec.rename(columns={col:renamefields[col]}, inplace=True)

In [83]:
if sntc is not None:
    for col in sntc.columns:
        if col in renamefields.keys():
            sntc.rename(columns={col:renamefields[col]}, inplace=True)

In [84]:
success.columns

Index(['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category', 'Sales Level 1',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'GUID Name', 'GUID',
       'CAV Name', 'CAV ID', 'Deal ID', 'Service Source', 'Partner Name',
       'CX Product Category', 'Product ID', 'TCV', 'ACV'],
      dtype='object')

In [85]:
if success is not None:
    success['AS Offer Type']='SuccessTracks'
    success['CX Product Category']='SuccessTracks'

<ipython-input-85-e24f17cd3040>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  success['AS Offer Type']='SuccessTracks'
<ipython-input-85-e24f17cd3040>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  success['CX Product Category']='SuccessTracks'


In [86]:
if sntc is not None:
    sntc['AS Offer Type']='SNTC'

In [87]:
if bcs is not None:
    bcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1703 entries, 0 to 1702
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  1703 non-null   int64  
 1   Fiscal Quarter ID            1703 non-null   object 
 2   Fiscal Period ID             1703 non-null   int64  
 3   Fiscal Week ID               1703 non-null   int64  
 4   Finance BU/Service Category  1703 non-null   object 
 5   Sales Level 1                1703 non-null   object 
 6   Sales Level 2                1703 non-null   object 
 7   Sales Level 3                1703 non-null   object 
 8   Sales Level 4                1703 non-null   object 
 9   GUID Name                    1703 non-null   object 
 10  GUID                         1703 non-null   int64  
 11  CAV Name                     1703 non-null   object 
 12  CAV ID                       1703 non-null   int64  
 13  Deal ID           

In [88]:
if htec is not None:
    htec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218023 entries, 0 to 218022
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Fiscal Year                  218023 non-null  int64  
 1   Fiscal Quarter ID            218023 non-null  object 
 2   Fiscal Period ID             218023 non-null  int64  
 3   Fiscal Week ID               218023 non-null  int64  
 4   Finance BU/Service Category  218023 non-null  object 
 5   Sales Level 1                218023 non-null  object 
 6   Sales Level 2                218023 non-null  object 
 7   Sales Level 3                218023 non-null  object 
 8   Sales Level 4                218023 non-null  object 
 9   GUID Name                    218023 non-null  object 
 10  GUID                         217789 non-null  object 
 11  CAV Name                     218023 non-null  object 
 12  CAV ID                       218023 non-null  int64  
 13 

In [89]:
if success is not None:
    success.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 1168 to 1713
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  546 non-null    int64  
 1   Fiscal Quarter ID            546 non-null    object 
 2   Fiscal Period ID             546 non-null    int64  
 3   Fiscal Week ID               546 non-null    int64  
 4   Finance BU/Service Category  546 non-null    object 
 5   Sales Level 1                546 non-null    object 
 6   Sales Level 2                546 non-null    object 
 7   Sales Level 3                546 non-null    object 
 8   Sales Level 4                546 non-null    object 
 9   GUID Name                    546 non-null    object 
 10  GUID                         546 non-null    int64  
 11  CAV Name                     546 non-null    object 
 12  CAV ID                       546 non-null    int64  
 13  Deal ID         

In [90]:
if success is not None and bcs is not None:
    success = success.reindex(columns=bcs.columns)

In [91]:
### Success Tracks is a cumulative query, so make sure this is limited to FY21 Q2

# success = success.loc[success['Fiscal Week ID'].isin([2021075,2021081])]

In [92]:
success.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 1168 to 1713
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  546 non-null    int64  
 1   Fiscal Quarter ID            546 non-null    object 
 2   Fiscal Period ID             546 non-null    int64  
 3   Fiscal Week ID               546 non-null    int64  
 4   Finance BU/Service Category  546 non-null    object 
 5   Sales Level 1                546 non-null    object 
 6   Sales Level 2                546 non-null    object 
 7   Sales Level 3                546 non-null    object 
 8   Sales Level 4                546 non-null    object 
 9   GUID Name                    546 non-null    object 
 10  GUID                         546 non-null    int64  
 11  CAV Name                     546 non-null    object 
 12  CAV ID                       546 non-null    int64  
 13  Deal ID         

In [93]:
# success

In [94]:
sntc = sntc.reindex(columns=bcs.columns)

In [95]:
sntc.shape

(249825, 21)

In [96]:
htec = htec.reindex(columns=bcs.columns)

In [97]:
df = pd.concat([bcs,htec,success, sntc])

In [ ]:
df = df.loc[(df['Fiscal Week ID']>= start_fiscal_week) & (df['Fiscal Week ID'] < end_fiscal_week)]

In [98]:
df['Fiscal Week ID'].unique()

array([2022071, 2022072, 2022073, 2022074, 2022081, 2022082, 2022083,
       2022084, 2022091, 2022092, 2022093, 2022094, 2022095], dtype=int64)

In [99]:
df['ReportWeek'] = df['Fiscal Week ID'].apply(get_reporting_week)

In [100]:
df.shape

(470097, 22)

In [101]:
new_column_list = []
for col in df.columns:
    if '\u00C2' in col:
        col = col.replace('\u00C2','').replace('\xa0',' ')
    elif '\xa0' in col:
        col = col.replace('\xa0',' ')
    new_column_list.append(col)

In [102]:
new_column_list

['Fiscal Year',
 'Fiscal Quarter ID',
 'Fiscal Period ID',
 'Fiscal Week ID',
 'Finance BU/Service Category',
 'Sales Level 1',
 'Sales Level 2',
 'Sales Level 3',
 'Sales Level 4',
 'GUID Name',
 'GUID',
 'CAV Name',
 'CAV ID',
 'Deal ID',
 'Service Source',
 'AS Offer Type',
 'Partner Name',
 'CX Product Category',
 'Product ID',
 'TCV',
 'ACV',
 'ReportWeek']

In [103]:
for i,element in enumerate(new_column_list):
    if element=='CAV End Customer ID':
        new_column_list[i] = 'CAV ID'
        
# new_column_list    

In [104]:
df.columns = new_column_list

In [105]:
df.columns

Index(['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category', 'Sales Level 1',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'GUID Name', 'GUID',
       'CAV Name', 'CAV ID', 'Deal ID', 'Service Source', 'AS Offer Type',
       'Partner Name', 'CX Product Category', 'Product ID', 'TCV', 'ACV',
       'ReportWeek'],
      dtype='object')

In [106]:
for col in df.columns:
    if col in renamefields.keys():
        df.rename(columns={col:renamefields[col]}, inplace=True)

In [107]:
df.rename(columns={'CAV End Customer ID':'CAV ID'}, inplace=True)

In [108]:
df['Deal ID'] = df['Deal ID'].apply(convert_nan_to_int)

In [109]:
df['CAV ID'] = df['CAV ID'].apply(convert_nan_to_int)

In [110]:
df['GUID'] = df['GUID'].apply(convert_nan_to_int)

In [111]:
def squeeze_customer_name(customer_name):
    if customer_name.startswith('THE '):
        customer_name = customer_name[4:]
    customer_name = customer_name.replace(',',"").replace('[ACTIVE]','').replace('-','').replace('_','').replace('.','').replace(' ','').strip()
    return customer_name

In [112]:
### Start with GUID. If GUID is None, use CAV ID, if CAV ID, is None, use CustomerName, if GUID is bad or a list with unacceptable alpha lexicon values, use CustomerName
def make_fake_guid(guid, cavid, customername):
    if pd.isnull(guid):
        if pd.isnull(cavid) or cavid == -999:
            return str(customername[:10])
        else:
            if not isinstance(cavid, int):
                cavid = cavid.split(',')
                return str(cavid[0])
    elif not isinstance(guid,int):
        guid = guid.split(',')
        return str(guid[0])
    else:
        return str(guid)

In [113]:
df['Squeezed'] = df['CAV Name'].apply(lambda x: squeeze_customer_name(x))

In [114]:
df['Prepend'] = df['Squeezed'].apply(lambda x: x[:2].upper())

In [115]:
df['FakeID'] = df.apply(lambda x: make_fake_guid(x['GUID'], x['CAV ID'], x['Squeezed']), axis=1)

In [116]:
df.loc[df['FakeID'].isnull()][['GUID','CAV ID','CAV Name', 'Squeezed','Prepend','FakeID']]

,GUID,CAV ID,CAV Name,Squeezed,Prepend,FakeID


In [117]:
#Prehash_Entity
df['Prehash_Entity'] = df.apply(lambda x: x['Prepend'] + x['FakeID'], axis=1)

In [118]:
df.columns

Index(['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category', 'Sales Level 1',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'GUID Name', 'GUID',
       'CAV Name', 'CAV ID', 'Deal ID', 'Service Source', 'AS Offer Type',
       'Partner Name', 'CX Product Category', 'Product ID', 'TCV', 'ACV',
       'ReportWeek', 'Squeezed', 'Prepend', 'FakeID', 'Prehash_Entity'],
      dtype='object')

In [119]:
# df['Deal ID'] = df['Deal ID'].fillna(value=-9999, axis = 0)

In [120]:
df = df[['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'ReportWeek', 'Finance BU/Service Category', 'Sales Level 1',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'Partner Name',
       'Deal ID', 'AS Offer Type', 'CX Product Category', 'Product ID', 'TCV', 'ACV', 'Service Source', 'Squeezed','Prepend','FakeID', 'Prehash_Entity']]

In [121]:
df.rename(columns = {'Deal ID':'FINBI Deal ID','Partner Name':'FINBI Partner'}, inplace=True)

In [122]:
# df.loc[df['CX Product Category'].str.lower()=='successtracks','CAV ID']=1

In [123]:
table = pd.pivot_table(df, values = ['ACV','TCV'], index=['Sales Level 1', 'Sales Level 2',
       'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name', 'GUID', 'GUID Name',  
       'FINBI Deal ID', 'FINBI Partner','AS Offer Type', 'CX Product Category', 'Fiscal Quarter ID','Fiscal Week ID', 'ReportWeek','Service Source','Squeezed','Prepend','FakeID', 'Prehash_Entity'], aggfunc='sum')

In [124]:
table.reset_index(inplace=True)

In [125]:
table['CX Product Category'].unique()

array(['SNTC', 'BCS 2.x', 'Solution Support', 'Expert Care', 'BCS 3.x',
       'SuccessTracks'], dtype=object)

In [126]:
table.sort_values(by=['CAV Name', 'GUID', 'GUID Name','FINBI Deal ID', 'Fiscal Week ID', 'AS Offer Type'], inplace=True)

In [127]:
tableP = table.loc[table['TCV']>0]

In [128]:
tableP['Ind'] = 'P'

<ipython-input-128-09d0920af551>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tableP['Ind'] = 'P'


In [129]:
tableP['AS Offer Type'].unique()

array(['Solution Support', 'SNTC', 'BCS', 'HTEC', 'SuccessTracks'],
      dtype=object)

In [130]:
tableP

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
12388,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_AUSTRIA,SMALL_AUT,650282,"""VOG"" EINFUHR UND GROBHANDEL MIT LEBENSMITTELN...",8441129,"""VOG"" EINFUHR UND GROBHANDEL MIT LEBENSMITTELN...",42171244,KAPSCH BUSINESSCOM AG,...,2022082,6,NEW,"""VOG""EINFUHRUNDGROBHANDELMITLEBENSMITTELNUNDBE...","""V",8441129,"""V8441129",22.20,66.60,P
22333,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_RUSSIA,COMMERCIAL_RUS,4911116,'ALLEGRO' COMPANI LIMITED,220262144,'ALLEGRO' COMPANI LIMITED,55719469,OLINS LLC INN 7705723717,...,2022074,4,NEW,'ALLEGRO'COMPANILIMITED,'A,220262144,'A220262144,"3,436.36","3,436.36",P
22334,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_RUSSIA,COMMERCIAL_RUS,4911116,'ALLEGRO' COMPANI LIMITED,220262144,'ALLEGRO' COMPANI LIMITED,55719469,OLINS LLC INN 7705723717,...,2022074,4,NEW,'ALLEGRO'COMPANILIMITED,'A,220262144,'A220262144,44.20,132.60,P
21081,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_RUSSIA,COMMERCIAL_RUS,123066,'ATON',222560259,'ATON',55716690,IBS PLATFORMIX LLC INN 7707507077,...,2022073,3,NEW,'ATON','A,222560259,'A222560259,"3,139.54","3,139.54",P
21082,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_RUSSIA,COMMERCIAL_RUS,123066,'ATON',222560259,'ATON',55716690,IBS PLATFORMIX LLC INN 7707507077,...,2022073,3,NEW,'ATON','A,222560259,'A222560259,81.96,245.88,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126000,EMEAR-REGION,EMEAR-UKI,SMALL_EAW,SBU_EAW_LONDON_L4,634415,ZYNGA INC,120034335,ZYNGA INC,53931341,TANGIBLE BENEFIT LTD,...,2022074,4,NEW,ZYNGAINC,ZY,120034335,ZY120034335,40.70,122.10,P
126001,EMEAR-REGION,EMEAR-UKI,SMALL_EAW,SBU_EAW_LONDON_L4,634415,ZYNGA INC,120034335,ZYNGA INC,53931341,TANGIBLE BENEFIT LTD,...,2022081,5,NEW,ZYNGAINC,ZY,120034335,ZY120034335,27.22,81.65,P
3847,EMEAR-REGION,EMEAR-CENTRAL,CLUSTER_BALKANS,CLUSTER_FYROM_KOS,7861412,ZYRA E REGULLATORIT PER ENERGJI,357879464,ZYRA E REGULLATORIT PER ENERGJI,53577580,InfoSoft Systems Sh.a,...,2022072,2,NEW,ZYRAEREGULLATORITPERENERGJI,ZY,357879464,ZY357879464,225.40,408.14,P
31848,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SMALL_CHE,1743508,ZZ VERMOEGENSBERATUNG SCHWEIZ AG,312499752,ZZ VERMOEGENSBERATUNG SCHWEIZ AG,-9999,VALITY AG,...,2022083,7,NEW,ZZVERMOEGENSBERATUNGSCHWEIZAG,ZZ,312499752,ZZ312499752,"3,416.08","3,416.08",P


In [131]:
tableN = table.loc[table['TCV']<=0]

In [132]:
tableN['Ind'] = 'N'

<ipython-input-132-7a79b8daff69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tableN['Ind'] = 'N'


In [133]:
tableN

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
12650,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_AUSTRIA,SMALL_AUT,1937368,"""HOTEL SCHLOSS MÖNCHSTEIN"" DOCIS GMBH & CO KG",217221501,"""HOTEL SCHLOSS M??CHSTEIN"" DOCIS GMBH & CO KG",-9999,A1 TELEKOM AUSTRIA AG,...,2022071,1,NEW,"""HOTELSCHLOSSMÖNCHSTEIN""DOCISGMBH&COKG","""H",217221501,"""H217221501",0.00,0.00,N
249497,EMEAR-REGION,EMEAR_SP,SP_SOUTH,TI_GROUP,1937368,"""HOTEL SCHLOSS MÖNCHSTEIN"" DOCIS GMBH & CO KG",217221501,"""HOTEL SCHLOSS M??CHSTEIN"" DOCIS GMBH & CO KG",-9999,A1 TELEKOM AUSTRIA AG,...,2022071,1,NEW,"""HOTELSCHLOSSMÖNCHSTEIN""DOCISGMBH&COKG","""H",217221501,"""H217221501",0.00,0.00,N
212689,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,257651,"""OSTERREICHISCHER AUTOMOBIL-, MOTORRAD- UND TO...",3969160,"""OSTERREICHISCHER AUTOMOBIL-, MOTORRAD- UND TO...",20721376,UNKNOWN,...,2022081,5,NEW,"""OSTERREICHISCHERAUTOMOBILMOTORRADUNDTOURINGCLUB""","""O",3969160,"""O3969160",0.00,0.00,N
212690,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,257651,"""OSTERREICHISCHER AUTOMOBIL-, MOTORRAD- UND TO...",3969160,"""OSTERREICHISCHER AUTOMOBIL-, MOTORRAD- UND TO...",53990476,UNKNOWN,...,2022074,4,NEW,"""OSTERREICHISCHERAUTOMOBILMOTORRADUNDTOURINGCLUB""","""O",3969160,"""O3969160",-48.26,-48.26,N
12387,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_AUSTRIA,SMALL_AUT,650282,"""VOG"" EINFUHR UND GROBHANDEL MIT LEBENSMITTELN...",8441129,"""VOG"" EINFUHR UND GROBHANDEL MIT LEBENSMITTELN...",42171244,KAPSCH BUSINESSCOM AG,...,2022081,5,NEW,"""VOG""EINFUHRUNDGROBHANDELMITLEBENSMITTELNUNDBE...","""V",8441129,"""V8441129",0.00,0.00,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3848,EMEAR-REGION,EMEAR-CENTRAL,CLUSTER_BALKANS,CLUSTER_FYROM_KOS,7861412,ZYRA E REGULLATORIT PER ENERGJI,357879464,ZYRA E REGULLATORIT PER ENERGJI,53577580,InfoSoft Systems Sh.a,...,2022073,3,NEW,ZYRAEREGULLATORITPERENERGJI,ZY,357879464,ZY357879464,-89.36,0.00,N
61457,EMEAR-REGION,EMEAR-NORTH,COUNTRY_SWEDEN,PUBLICSECTOR_SWE_REGIONAL,351301,ÖREBROBOSTÄDER AB,4341454,??EBROBOST??ER AB,53000755,ATEA ASA,...,2022083,7,NEW,ÖREBROBOSTÄDERAB,ÖR,4341454,ÖR4341454,0.00,0.00,N
6317,EMEAR-REGION,EMEAR-CENTRAL,CLUSTER_CIS,COUNTRY_KAZAKHSTAN,3369614,КAZAKHSTAN ELECTRICITYGRID OPERATING COMPANY,359569651,AZAKHSTAN ELECTRICITYGRID OPERATING COMPANY,21013436,Asia Intercommunications,...,2022071,1,NEW,КAZAKHSTANELECTRICITYGRIDOPERATINGCOMPANY,КA,359569651,КA359569651,0.00,0.00,N
237244,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,3369614,КAZAKHSTAN ELECTRICITYGRID OPERATING COMPANY,359569651,AZAKHSTAN ELECTRICITYGRID OPERATING COMPANY,21013436,UNKNOWN,...,2022071,1,NEW,КAZAKHSTANELECTRICITYGRIDOPERATINGCOMPANY,КA,359569651,КA359569651,0.00,0.00,N


In [134]:
### This is DROP Criteria; i.e. > -10,000 to 0

tableN.drop(tableN.loc[(tableN['TCV'] > -10000) & (tableN['TCV'] <= 0)].index, inplace=True)   #tableN.drop(((tableN['TCV'] > -20000) & (tableN['TCV'] <= 0))].index, inplace=True) 

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [135]:
tableN

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
104455,EMEAR-REGION,EMEAR-SOUTH,SMALL_SOUTH,SMALL_ITA,2798771,AB MEDICA SPA,43384665,AB MEDICA SPA,-9999,ITCore Business Group srl,...,2022081,5,NEW,ABMEDICASPA,AB,43384665,AB43384665,"-9,459.28","-14,639.50",N
202307,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,107999,AB SKF,101280,AB SKF,43906229,UNKNOWN,...,2022071,1,NEW,ABSKF,AB,101280,AB101280,"-18,888.74","-53,518.09",N
204309,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,112193,ABB LTD,4260091,ABB LTD,44214101,UNKNOWN,...,2022092,10,RENEW,ABBLTD,AB,4260091,AB4260091,"-12,274.87","-12,274.87",N
204492,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,112193,ABB LTD,4260091,ABB LTD,54021456,UNKNOWN,...,2022083,7,NEW,ABBLTD,AB,4260091,AB4260091,"-22,070.15","-22,070.15",N
204498,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,112193,ABB LTD,4260091,ABB LTD,54041736,UNKNOWN,...,2022093,11,NEW,ABBLTD,AB,4260091,AB4260091,"-2,982.90","-14,914.51",N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33406,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,52212205,TECHNOLOGENT,...,2022081,5,NEW,ZURICHINSURANCEGROUPAG,ZU,4892203,ZU4892203,"-6,879.45","-10,126.75",N
33435,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506130,BT,...,2022093,11,RENEW,ZURICHINSURANCEGROUPAG,ZU,4892203,ZU4892203,"-15,724.53","-15,724.53",N
33443,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506136,BT,...,2022093,11,RENEW,ZURICHINSURANCEGROUPAG,ZU,4892203,ZU4892203,"-19,270.22","-19,270.22",N
33449,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506136,BT,...,2022093,11,RENEW,ZURICHINSURANCEGROUPAG,ZU,4892203,ZU4892203,"-48,108.21","-48,108.21",N


In [136]:
# tableP.reset_index(inplace=True)

In [137]:
bcs_condition = ((tableP['TCV']>= 10000) & (tableP['AS Offer Type']=='BCS'))
htec_condition = ((tableP['TCV']>= 10000) & (tableP['AS Offer Type']=='HTEC'))
ssspt_condition = ((tableP['TCV']>= 10000) & (tableP['AS Offer Type']=='Solution Support'))
st_condition = ((tableP['TCV']>= 500) & (tableP['AS Offer Type']=='SuccessTracks'))

In [138]:
tableP.loc[bcs_condition | htec_condition | ssspt_condition | st_condition]

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
103925,EMEAR-REGION,EMEAR-SOUTH,SMALL_SOUTH,SMALL_ITA,1748986,3F FILIPPI SPA,6470243,3F FILIPPI SPA,53917021,LOGOS TECHNOLOGIES S.R.L.,...,2022091,9,NEW,3FFILIPPISPA,3F,6470243,3F6470243,"3,495.54","10,486.61",P
34865,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,112193,ABB LTD,4260091,ABB LTD,44214101,BT,...,2022092,10,RENEW,ABBLTD,AB,4260091,AB4260091,"87,117.58","87,117.58",P
35317,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,112193,ABB LTD,4260091,ABB LTD,54021456,BT,...,2022083,7,NEW,ABBLTD,AB,4260091,AB4260091,"7,984.57","23,953.71",P
35324,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,112193,ABB LTD,4260091,ABB LTD,54041736,BT,...,2022093,11,NEW,ABBLTD,AB,4260091,AB4260091,"2,598.21","12,991.04",P
105386,EMEAR-REGION,EMEAR-UKI,CLUSTER_IRELAND_SCOTLAND,COMMERCIAL_IRL,112202,ABBOTT LABORATORIES,10368,ABBOTT LABORATORIES,-9999,Dell,...,2022082,6,NEW,ABBOTTLABORATORIES,AB,10368,AB10368,"12,849.81","64,249.07",P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19013,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_POLAND,PUBLIC_SECTOR_POL,169927,ZAKLAD UBEZPIECZEN SPOLECZNYCH,4644472,ZAKLAD UBEZPIECZEN SPOLECZNYCH,55681976,UpWare Sp. z o.o.,...,2022074,4,NEW,ZAKLADUBEZPIECZENSPOLECZNYCH,ZA,4644472,ZA4644472,"12,777.65","39,397.75",P
21303,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_RUSSIA,COMMERCIAL_RUS,241936,ZAO VECTOR-BEST,142096292,ZAO VECTOR-BEST,55686868,Toliman CS LLC INN 5408016503,...,2022072,2,NEW,ZAOVECTORBEST,ZA,142096292,ZA142096292,"11,406.86","57,034.32",P
11581,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_AUSTRIA,PUBLIC_SECTOR_AUT,342510,ZENTRALVERWALTUNG DER BARMHERZIGEN BRUDER AUSTRIA,35586378,ZENTRALVERWALTUNG DER BARMHERZIGEN BRUDER AUSTRIA,53075874,A1 TELEKOM AUSTRIA AG,...,2022074,4,RENEW,ZENTRALVERWALTUNGDERBARMHERZIGENBRUDERAUSTRIA,ZE,35586378,ZE35586378,"1,530.24","10,711.65",P
54316,EMEAR-REGION,EMEAR-NORTH,COUNTRY_NETHERLANDS,ENTERPRISE_NLD,688479,ZUIVELCOOEPERATIE FRIESLANDCAMPINA UA,2960205,ZUIVELCOOEPERATIE FRIESLANDCAMPINA UA,53689456,AXIANS,...,2022081,5,NEW,ZUIVELCOOEPERATIEFRIESLANDCAMPINAUA,ZU,2960205,ZU2960205,"6,453.99","32,269.96",P


In [139]:
tableP.loc[tableP['AS Offer Type'].str.contains('Success', case=False)]

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
167831,EMEAR-REGION,EMEAR_MEA,CLUSTER_GULF,CLUSTER_REST_OF_GULF,7898017,BAHRAIN TOURISM AND EXHIBITION AUTHORITY,305636708,BAHRAIN TOURISM AND EXHIBITION AUTHORITY,52670290,UNKNOWN,...,2022092,10,NEW,BAHRAINTOURISMANDEXHIBITIONAUTHORITY,BA,305636708,BA305636708,"2,298.81","6,896.43",P
20043,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_ROMANIA,COMMERCIAL_ROU,645120,COOP-GRUPPE GENOSSENSCHAFT,2383448,COOP-GRUPPE GENOSSENSCHAFT,55983381,Arctic Stream S.A.,...,2022091,9,NEW,COOPGRUPPEGENOSSENSCHAFT,CO,2383448,CO2383448,"2,226.03","4,266.55",P
20044,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_ROMANIA,COMMERCIAL_ROU,645120,COOP-GRUPPE GENOSSENSCHAFT,2383448,COOP-GRUPPE GENOSSENSCHAFT,55983381,Arctic Stream S.A.,...,2022091,9,RENEW,COOPGRUPPEGENOSSENSCHAFT,CO,2383448,CO2383448,870.26,"1,668.00",P
8149,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_AUSTRIA,COMMERCIAL_AUT,115234,CSL LTD,7251413,CSL LTD,44567760,CDW,...,2022092,10,NEW,CSLLTD,CS,7251413,CS7251413,3.80,10.74,P
8150,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_AUSTRIA,COMMERCIAL_AUT,115234,CSL LTD,7251413,CSL LTD,44567760,CDW,...,2022092,10,RENEW,CSLLTD,CS,7251413,CS7251413,4.49,12.65,P
13093,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_CZECH REPUBLIC,PRIVATE_SECTOR_CZE_1,115234,CSL LTD,7251413,CSL LTD,44567760,CDW,...,2022092,10,NEW,CSLLTD,CS,7251413,CS7251413,3.80,10.74,P
13094,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_CZECH REPUBLIC,PRIVATE_SECTOR_CZE_1,115234,CSL LTD,7251413,CSL LTD,44567760,CDW,...,2022092,10,RENEW,CSLLTD,CS,7251413,CS7251413,4.49,12.65,P
15748,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_HUNGARY,PRIVATE_SECTOR_HUN,115234,CSL LTD,7251413,CSL LTD,44567760,CDW,...,2022092,10,NEW,CSLLTD,CS,7251413,CS7251413,18.99,53.56,P
15749,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_HUNGARY,PRIVATE_SECTOR_HUN,115234,CSL LTD,7251413,CSL LTD,44567760,CDW,...,2022092,10,RENEW,CSLLTD,CS,7251413,CS7251413,22.47,63.06,P
25647,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,COMMERCIAL_CHE,115234,CSL LTD,7251413,CSL LTD,44567760,CDW,...,2022092,10,NEW,CSLLTD,CS,7251413,CS7251413,"6,573.21","18,548.16",P


In [140]:
tableP = tableP.loc[bcs_condition | htec_condition | ssspt_condition | st_condition]

In [141]:
table = pd.concat([tableN,tableP])

In [142]:
# table = table.loc[~(table['Deal ID']==-9999)]

In [143]:
# table

In [144]:
table.sort_values(by=['CAV Name', 'GUID', 'GUID Name','FINBI Deal ID', 'Fiscal Week ID', 'AS Offer Type', 'Ind'], inplace=True)

In [145]:
table['CX Product Category'].unique()

array(['Solution Support', 'SNTC', 'BCS 2.x', 'Expert Care', 'BCS 3.x',
       'SuccessTracks'], dtype=object)

### Add Readouts to the table

In [146]:
def get_minimum_date_from_weekid(weekid):
    return cal.loc[cal['finbi_fiscal_week_id']==weekid, 'date'].max()

In [147]:
fiscalweeks = list(table['Fiscal Week ID'].unique())

In [148]:
fiscalweeks.sort()

In [149]:
cal = cal.loc[cal['finbi_fiscal_week_id'].isin(fiscalweeks)]

In [150]:
### Add a time delta function here to get back the 30 days PRIOR to this min date for each fiscal week
min_dates_dict = {}
for x in fiscalweeks:
    min_dates_dict[x] = get_minimum_date_from_weekid(x)

min_dates_dict

{2022071: '2022-02-05',
 2022072: '2022-02-12',
 2022073: '2022-02-19',
 2022074: '2022-02-26',
 2022081: '2022-03-05',
 2022082: '2022-03-12',
 2022083: '2022-03-19',
 2022084: '2022-03-26',
 2022091: '2022-04-02',
 2022092: '2022-04-09',
 2022093: '2022-04-16'}

In [151]:
must_be_prior_to_dict = {}
for k, v in min_dates_dict.items():
    must_be_prior_to_dict[k] = (pd.to_datetime(v) + dt.timedelta(days = -30))

In [152]:
must_be_prior_to_dict

{2022071: Timestamp('2022-01-06 00:00:00'),
 2022072: Timestamp('2022-01-13 00:00:00'),
 2022073: Timestamp('2022-01-20 00:00:00'),
 2022074: Timestamp('2022-01-27 00:00:00'),
 2022081: Timestamp('2022-02-03 00:00:00'),
 2022082: Timestamp('2022-02-10 00:00:00'),
 2022083: Timestamp('2022-02-17 00:00:00'),
 2022084: Timestamp('2022-02-24 00:00:00'),
 2022091: Timestamp('2022-03-03 00:00:00'),
 2022092: Timestamp('2022-03-10 00:00:00'),
 2022093: Timestamp('2022-03-17 00:00:00')}

In [153]:
ro['Readout_Date'] = pd.to_datetime(ro['Readout_Date'])

<ipython-input-153-909d594d0e5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ro['Readout_Date'] = pd.to_datetime(ro['Readout_Date'])


In [154]:
ro.loc[ro['Readout_Date'].isnull()][['Request ID','Readout_Date']]

,Request ID,Readout_Date


In [155]:
ro['Readout_Date'] = ro['Readout_Date'].astype(str).replace('-','')

<ipython-input-155-b82cb0de2393>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ro['Readout_Date'] = ro['Readout_Date'].astype(str).replace('-','')


In [156]:
ro = ro.loc[(ro['Readout_Date'].notnull()) & (ro['Sales Level 1']=='EMEAR-REGION') & ~(ro['Readout Delivery Status'].isin(['Declined','Postponed']))]

In [157]:
ro.loc[ro['CONCIERGE_INTERNAL_READOUT_DT'].notnull()]

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
324,114609,146930,EMEAR-REGION,EMEAR-NORTH,CLUSTER_BELUX,COMMERCIAL_BEL,AGC,55998345,COVERAGE PROGRAM - TELEMETRY,Viriato Evangelista,AG146930,2022-03-23,Complete,"BCS,DNA,BRW,EC-AM,EC-KT,IRR,PR,SNTC,ST",COVERAGE PROGRAM,glcanale@cisco.com,2022-03-25
948,116046,153717407,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_GULF,NETWORK INTERNATIONAL LLC,<NA>,REACTIVE,Camila Bernal,NE153717407,2022-03-17,Complete,"EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST,BCS",FIELD,macoup@cisco.com,2022-03-17
1160,116410,77771,EMEAR-REGION,EMEAR-SOUTH,COUNTRY_FRANCE,CGEM_FRA,L'OREAL - APJC,<NA>,REACTIVE,Viriato Evangelista,L'77771,2022-04-20,NaN,NaN,FIELD,macoup@cisco.com,2022-04-20
2023,107954,4251796,EMEAR-REGION,EMEAR_GERMANY,ENTERPRISE_II_DEU,ENT_DEU_SOUTH,INFINEON TECHNOLOGIES AG,20363258,REACTIVE,Martin Coup,IN4251796,2021-12-13,Complete,"BCS,DNA,EC-HTOM,EC-KT,IRR,SNTC",FIELD,macoup@cisco.com,2021-12-13
2028,107966,70684,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW,ENT_EAW_G_T_INDUSTRIAL,ASTRAZENECA PLC,20725097,REACTIVE,Atif Ahmad,AS70684,2021-12-09,Complete,NaN,FIELD,atahmad@cisco.com,2022-01-12
2773,108770,2128073,EMEAR-REGION,EMEAR-SOUTH,COUNTRY_FRANCE,ENTERPRISE_FRA,BOUYGUES TELECOM,<NA>,REACTIVE,Viriato Evangelista,BO2128073,2022-01-19,Complete,"BCS,BRW,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SSPT,ST",FIELD,atahmad@cisco.com,2022-01-19
3104,110443,2389367,EMEAR-REGION,EMEAR_MEA,COUNTRY_KSA,ENTERPRISE_KSA,ARAB NATIONAL BANK,<NA>,COVERAGE PROGRAM - TELEMETRY,Camila Bernal,AR2389367,2022-01-24,Complete,"BCS,ST,SSPT,SNTC,DNA",COVERAGE PROGRAM,atahmad@cisco.com,2022-02-17
3404,110731,37440280,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_AUSTRIA,SMALL_AUT,LINZ AG FUR ENERGIE TELEKOMMUNIKATION VERKEHR ...,<NA>,REACTIVE,Viriato Evangelista,LI37440280,2022-02-03,Complete,"BCS,BRW,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,atahmad@cisco.com,2022-02-03
4684,111969,34694163,EMEAR-REGION,EMEAR_MEA,CLUSTER_GULF,CLUSTER_REST_OF_GULF,SOHAR ALUMINIUM,<NA>,REACTIVE,Camila Bernal,SO34694163,2022-02-28,Complete,"ST,SSPT,IRR",FIELD,atahmad@cisco.com,2022-02-28
4811,112070,2120842,EMEAR-REGION,EMEAR-UKI,PUBLIC_SECTOR_EAW,PS_EAW_DGOV,NERC,44337491,REACTIVE,Martin Coup,NE2120842,2022-03-07,Complete,"BRW,SNTC,SSPT",FIELD,macoup@cisco.com,2022-03-07


In [158]:
ro['Readout_Date'] = pd.to_datetime(ro['Readout_Date'])
ro['CONCIERGE_INTERNAL_READOUT_DT'] = pd.to_datetime(ro['CONCIERGE_INTERNAL_READOUT_DT'])

In [159]:
MaxList = pd.DataFrame(ro.groupby('Compass GUID Name')['Readout_Date'].min())  #should be minimum date to get the best possible date differential

In [160]:
MaxList.reset_index(inplace=True)

In [161]:
MaxList

,Compass GUID Name,Readout_Date
0,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022-01-10
1,(PIC) PUBLIC INVESTMENT CORPORATION,2022-01-24
2,(SAA) SOUTH AFRICAN AIRWAYS,2022-01-10
3,(SAPS) – SOUTH AFRICAN POLICE SERVICES,2022-01-24
4,(SARS) SOUTH AFRICAN RESERVE SERVICES,2022-01-24
...,...,...
913,ZEB ROLFESSCHIERENBECKASSOCIATES GMBH,2021-11-12
914,ZOLLERN VERTRIEBS-GMBH COMPANY KG,2022-03-01
915,ZUCCHETTI GROUP SPA,2022-04-19
916,ZURICH INSURANCE GROUP,2022-01-19


In [162]:
readouts = pd.merge(ro,MaxList, how='inner', left_on = 'Compass GUID Name', right_on='Compass GUID Name').drop(columns = ['Readout_Date_y'])#.rename(columns={'Readout_Date_x':'Readout_Date'}, inplace=True)

In [163]:
readouts.rename(columns={'Readout_Date_x':'Readout_Date'}, inplace=True)

In [164]:
def set_threshold_date(finbi_week_id):
    return must_be_prior_to_dict.get(finbi_week_id)

In [165]:
table['Threshold Date'] = table.apply(lambda x: set_threshold_date(x['Fiscal Week ID']), axis=1)

In [166]:
table

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind,Threshold Date
103925,EMEAR-REGION,EMEAR-SOUTH,SMALL_SOUTH,SMALL_ITA,1748986,3F FILIPPI SPA,6470243,3F FILIPPI SPA,53917021,LOGOS TECHNOLOGIES S.R.L.,...,9,NEW,3FFILIPPISPA,3F,6470243,3F6470243,"3,495.54","10,486.61",P,2022-03-03
104455,EMEAR-REGION,EMEAR-SOUTH,SMALL_SOUTH,SMALL_ITA,2798771,AB MEDICA SPA,43384665,AB MEDICA SPA,-9999,ITCore Business Group srl,...,5,NEW,ABMEDICASPA,AB,43384665,AB43384665,"-9,459.28","-14,639.50",N,2022-02-03
202307,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,107999,AB SKF,101280,AB SKF,43906229,UNKNOWN,...,1,NEW,ABSKF,AB,101280,AB101280,"-18,888.74","-53,518.09",N,2022-01-06
204309,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,112193,ABB LTD,4260091,ABB LTD,44214101,UNKNOWN,...,10,RENEW,ABBLTD,AB,4260091,AB4260091,"-12,274.87","-12,274.87",N,2022-03-10
34865,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,112193,ABB LTD,4260091,ABB LTD,44214101,BT,...,10,RENEW,ABBLTD,AB,4260091,AB4260091,"87,117.58","87,117.58",P,2022-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33406,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,52212205,TECHNOLOGENT,...,5,NEW,ZURICHINSURANCEGROUPAG,ZU,4892203,ZU4892203,"-6,879.45","-10,126.75",N,2022-02-03
33435,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506130,BT,...,11,RENEW,ZURICHINSURANCEGROUPAG,ZU,4892203,ZU4892203,"-15,724.53","-15,724.53",N,2022-03-17
33443,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506136,BT,...,11,RENEW,ZURICHINSURANCEGROUPAG,ZU,4892203,ZU4892203,"-19,270.22","-19,270.22",N,2022-03-17
33449,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506136,BT,...,11,RENEW,ZURICHINSURANCEGROUPAG,ZU,4892203,ZU4892203,"-48,108.21","-48,108.21",N,2022-03-17


In [167]:
readouts.sort_values(by=['Compass GUID Name','Readout_Date'], inplace=True)

In [168]:
readouts.drop_duplicates(subset=['Compass GUID Name'], keep='last', inplace=True)

In [169]:
readouts

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
549,108132,30055,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,<NA>,REACTIVE,Thiago Rocha,(A9560192,2022-01-10,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
551,108133,186250284,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,(PIC) PUBLIC INVESTMENT CORPORATION,<NA>,REACTIVE,Thiago Rocha,(P186250284,2022-01-24,Complete,SNTC,FIELD,NaN,NaT
545,108131,181530329,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,(SAA) SOUTH AFRICAN AIRWAYS,<NA>,REACTIVE,Thiago Rocha,(S8888171,2022-01-10,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
540,108119,8888171,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,(SAPS) – SOUTH AFRICAN POLICE SERVICES,<NA>,REACTIVE,Thiago Rocha,(S8888171,2022-01-24,Complete,SNTC,FIELD,NaN,NaT
550,108130,41596700,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_SSA,(SARS) SOUTH AFRICAN RESERVE SERVICES,<NA>,REACTIVE,Thiago Rocha,(S41596700,2022-01-24,Complete,SNTC,FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,104950,8650158,EMEAR-REGION,EMEAR_GERMANY,COMMERCIAL_DEU,COM_DEU_WEST,ZEB ROLFESSCHIERENBECKASSOCIATES GMBH,55891001,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,Viriato Evangelista,ZE8650158,2021-11-12,Complete,SNTC,COVERAGE PROGRAM,NaN,NaT
720,110108,4250016,EMEAR-REGION,EMEAR_GERMANY,COMMERCIAL_DEU,COM_DEU_MIDSIZE,ZOLLERN VERTRIEBS-GMBH COMPANY KG,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Martin Coup,ZO4250016,2022-03-01,Complete,SNTC,COVERAGE AT RENEWAL,NaN,NaT
145,116025,197616515,EMEAR-REGION,EMEAR-SOUTH,COUNTRY_ITALY,COMMERCIAL_ITA,ZUCCHETTI GROUP SPA,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Viriato Evangelista,ZU197616515,2022-04-19,NaN,NaN,COVERAGE AT RENEWAL,NaN,NaT
1404,101609,4892203,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,ZURICH INSURANCE GROUP,44476401,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Martin Coup,ZU4892203,2022-01-19,Complete,"BCS,BRW,DNA,EC-KT,IRR,ST",TARGETING,NaN,NaT


In [170]:
compass_exists_table = pd.merge(table,readouts, how='left', left_on=['Prehash_Entity'], right_on = ['Prehash_Entity'])    # ,'CAV Name'       #,'Compass GUID Name'

In [171]:
compass_exists_table.shape

(3363, 40)

In [172]:
compass_exists_table

,Sales Level 1_x,Sales Level 2_x,Sales Level 3_x,Sales Level 4_x,CAV ID,CAV Name,GUID_x,GUID Name,FINBI Deal ID,FINBI Partner,...,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,EMEAR-REGION,EMEAR-SOUTH,SMALL_SOUTH,SMALL_ITA,1748986,3F FILIPPI SPA,6470243,3F FILIPPI SPA,53917021,LOGOS TECHNOLOGIES S.R.L.,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
1,EMEAR-REGION,EMEAR-SOUTH,SMALL_SOUTH,SMALL_ITA,2798771,AB MEDICA SPA,43384665,AB MEDICA SPA,-9999,ITCore Business Group srl,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
2,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,107999,AB SKF,101280,AB SKF,43906229,UNKNOWN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
3,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,112193,ABB LTD,4260091,ABB LTD,44214101,UNKNOWN,...,ABB LTD,43638739,REACTIVE,Camila Bernal,2021-10-15,Complete,"BCS,EC-KT,EC-HTOM,EC-AM,IRR,SNTC,SSPT,ST,PR,BRW",FIELD,NaN,NaT
4,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,112193,ABB LTD,4260091,ABB LTD,44214101,BT,...,ABB LTD,43638739,REACTIVE,Camila Bernal,2021-10-15,Complete,"BCS,EC-KT,EC-HTOM,EC-AM,IRR,SNTC,SSPT,ST,PR,BRW",FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3358,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506136,BT,...,ZURICH INSURANCE GROUP,44476401,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Martin Coup,2022-01-19,Complete,"BCS,BRW,DNA,EC-KT,IRR,ST",TARGETING,NaN,NaT
3359,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506136,BT,...,ZURICH INSURANCE GROUP (ZIG),43655289,REACTIVE,Martin Coup,2022-01-19,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,ST",FIELD,NaN,NaT
3360,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506136,BT,...,ZURICH INSURANCE GROUP,44476401,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Martin Coup,2022-01-19,Complete,"BCS,BRW,DNA,EC-KT,IRR,ST",TARGETING,NaN,NaT
3361,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,53506136,BT,...,ZURICH INSURANCE GROUP (ZIG),43655289,REACTIVE,Martin Coup,2022-01-19,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,ST",FIELD,NaN,NaT


In [173]:
compass_exists_table.rename(columns = {'GUID Name_x':'GUID Name','GUID Name_y':'Compass GUID Name'}, inplace=True)

In [174]:
compass_exists_table.columns

Index(['Sales Level 1_x', 'Sales Level 2_x', 'Sales Level 3_x',
       'Sales Level 4_x', 'CAV ID', 'CAV Name', 'GUID_x', 'GUID Name',
       'FINBI Deal ID', 'FINBI Partner', 'AS Offer Type',
       'CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID',
       'ReportWeek', 'Service Source', 'Squeezed', 'Prepend', 'FakeID',
       'Prehash_Entity', 'ACV', 'TCV', 'Ind', 'Threshold Date', 'Request ID',
       'GUID_y', 'Sales Level 1_y', 'Sales Level 2_y', 'Sales Level 3_y',
       'Sales Level 4_y', 'Compass GUID Name', 'Compass Deal ID', 'Campaign',
       'Compass Advisor', 'Readout_Date', 'Readout Delivery Status',
       'Items Recommended', 'Campaign Type', 'CONCIERGE_ADVISOR_ASSIGNED',
       'CONCIERGE_INTERNAL_READOUT_DT'],
      dtype='object')

In [175]:
compass_exists_table.rename(columns={'Sales Level 1_x':'Sales Level 1',
                                     'Sales Level 2_x':'Sales Level 2',
                                     'Sales Level 3_x':'Sales Level 3',
                                     'Sales Level 4_x':'Sales Level 4',
                                     'GUID_x':'GUID',
                                     'GUID_y':'Compass GUID'}, inplace=True)

compass_exists_table.drop(columns=(['Squeezed', 'Prepend','FakeID', 'Prehash_Entity','Sales Level 1_y','Sales Level 2_y', 'Sales Level 3_y','Sales Level 4_y',]), inplace=True)

In [176]:
# compass_exists_table = pd.merge(compass_exists_table, services_source, how='left', left_on = 'FINBI Deal ID', right_on = 'Deal ID')

In [177]:
compass_exists_table['Readout_Date'] = pd.to_datetime(compass_exists_table['Readout_Date'])

In [178]:
compass_exists_table['Threshold Date'] = pd.to_datetime(compass_exists_table['Threshold Date'])

In [179]:
compass_exists_table['CONCIERGE_INTERNAL_READOUT_DT'] = pd.to_datetime(compass_exists_table['CONCIERGE_INTERNAL_READOUT_DT'])

In [180]:
condition1 = ((compass_exists_table['CONCIERGE_INTERNAL_READOUT_DT'] <= compass_exists_table['Threshold Date']) | (compass_exists_table['Readout_Date'] <= compass_exists_table['Threshold Date']))  #
condition2 = (compass_exists_table['Readout_Date'].notnull()) #& (compass_exists_table['FINBI Deal ID']==-9999))

In [181]:
compass_exists_table = compass_exists_table.loc[condition1 & condition2] #[['FINBI Deal ID','Threshold Date','Readout_Date', 'Compass Deal ID', 'Deal ID']]

In [182]:
product_types = {'BCS 2.x':'BCS 2',
                 'BCS 2.0':'BCS 2',
                 'BCS 3.x':'BCS 3',
                 'BCS 3.0':'BCS 3',
                 'HTEC':'HTEC',
                 'Expert Care':'HTEC',
                 'Level 1':'ST',
                 'Level 2':'ST',
                 'Solution Support':'SSPT',
                 'SuccessTracks':'ST',
                 'Campus Network':'ST',
                 'Unknown':'UNK',
                 'SNTC':'SNTC'}

In [183]:
compass_exists_table.rename(columns={'AS Offer Type':'Product Type'}, inplace=True)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [184]:
compass_exists_table['Compass Deal ID'] = compass_exists_table['Compass Deal ID'].apply(convert_nan_to_int)

<ipython-input-184-e2b0b5ca6b42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Compass Deal ID'] = compass_exists_table['Compass Deal ID'].apply(convert_nan_to_int)


In [185]:
compass_exists_table['Product Type'] = compass_exists_table['CX Product Category'].apply(lambda x: product_types.get(x))

<ipython-input-185-dcc40366fa47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Product Type'] = compass_exists_table['CX Product Category'].apply(lambda x: product_types.get(x))


In [186]:
compass_exists_table.sort_values(by=['CAV Name', 'GUID Name', 'FINBI Deal ID', 'Fiscal Quarter ID', 'ReportWeek', 'Sales Level 1','Sales Level 2','Sales Level 3', 'Sales Level 4','Product Type','TCV'], inplace=True)

<ipython-input-186-12d2f74770a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table.sort_values(by=['CAV Name', 'GUID Name', 'FINBI Deal ID', 'Fiscal Quarter ID', 'ReportWeek', 'Sales Level 1','Sales Level 2','Sales Level 3', 'Sales Level 4','Product Type','TCV'], inplace=True)


In [187]:
def apply_targeting_campaign_flag(compass_campaign_name):
    if pd.notnull(compass_campaign_name) and ('target' in compass_campaign_name.lower() or 'bcs' in compass_campaign_name.lower() or 'major' in compass_campaign_name.lower()  or 'success' in compass_campaign_name.lower()):
        return True
    else:
        return False

In [188]:
compass_exists_table['Target Campaign Flag'] = compass_exists_table.apply(lambda x: apply_targeting_campaign_flag(x['Campaign']), axis=1)

<ipython-input-188-cc7ead0f3f70>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Target Campaign Flag'] = compass_exists_table.apply(lambda x: apply_targeting_campaign_flag(x['Campaign']), axis=1)


In [189]:
compass_exists_table['DupeCheck'] =  compass_exists_table.duplicated(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name','GUID', 'GUID Name', 'FINBI Partner', 'FINBI Deal ID','CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID','ReportWeek', 'Product Type', 'TCV'], keep=False)

<ipython-input-189-9418a8a9949c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['DupeCheck'] =  compass_exists_table.duplicated(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name','GUID', 'GUID Name', 'FINBI Partner', 'FINBI Deal ID','CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID','ReportWeek', 'Product Type', 'TCV'], keep=False)


In [190]:
compass_exists_table['Compass Exists?'] = ''
compass_exists_table['Compass Claim?'] = ''
compass_exists_table['Notes'] = ''

<ipython-input-190-0c0e975f3ae4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Compass Exists?'] = ''
<ipython-input-190-0c0e975f3ae4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Compass Claim?'] = ''
<ipython-input-190-0c0e975f3ae4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [191]:
compass_exists_table = compass_exists_table[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Partner', 'FINBI Deal ID',
                                             'CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID', 'ReportWeek', 'Product Type', 'TCV', 'ACV', 'Ind', 'DupeCheck', 
                                             'Request ID', 'Compass GUID','Compass GUID Name', 'Compass Deal ID', 'Campaign', 'Threshold Date', 'Readout_Date',
                                             'Compass Advisor', 'Readout Delivery Status', 'Items Recommended', 'Campaign Type', 'Service Source', 'CONCIERGE_ADVISOR_ASSIGNED', 'CONCIERGE_INTERNAL_READOUT_DT','Target Campaign Flag', 'Compass Exists?', 
                                             'Compass Claim?', 'Notes', 
                                            ]]

In [192]:
compass_exists_table = compass_exists_table.loc[(compass_exists_table['Readout Delivery Status']!= 'Declined')] 

In [193]:
# compass_exists_table = compass_exists_table.loc[(compass_exists_table['Service Source'].notnull())]

In [194]:
table.rename(columns={'AS Offer Type':'Product Type'}, inplace=True)

In [195]:
table['CX Product Category'].unique()

array(['Solution Support', 'SNTC', 'BCS 2.x', 'Expert Care', 'BCS 3.x',
       'SuccessTracks'], dtype=object)

In [196]:
table['Product Type'] = table['CX Product Category'].apply(lambda x: product_types.get(x))

In [197]:
## Check the Targeting Campaign Flag Attribution
# compass_exists_table[['Campaign','Target Campaign Flag']].drop_duplicates(keep='first')
compass_exists_table

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Partner,FINBI Deal ID,...,Readout Delivery Status,Items Recommended,Campaign Type,Service Source,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT,Target Campaign Flag,Compass Exists?,Compass Claim?,Notes
4,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,112193,ABB LTD,4260091,ABB LTD,BT,44214101,...,Complete,"BCS,EC-KT,EC-HTOM,EC-AM,IRR,SNTC,SSPT,ST,PR,BRW",FIELD,RENEW,NaN,NaT,False,,,
3,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,112193,ABB LTD,4260091,ABB LTD,UNKNOWN,44214101,...,Complete,"BCS,EC-KT,EC-HTOM,EC-AM,IRR,SNTC,SSPT,ST,PR,BRW",FIELD,RENEW,NaN,NaT,False,,,
6,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,112193,ABB LTD,4260091,ABB LTD,BT,54021456,...,Complete,"BCS,EC-KT,EC-HTOM,EC-AM,IRR,SNTC,SSPT,ST,PR,BRW",FIELD,NEW,NaN,NaT,False,,,
5,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,112193,ABB LTD,4260091,ABB LTD,UNKNOWN,54021456,...,Complete,"BCS,EC-KT,EC-HTOM,EC-AM,IRR,SNTC,SSPT,ST,PR,BRW",FIELD,NEW,NaN,NaT,False,,,
8,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,112193,ABB LTD,4260091,ABB LTD,BT,54041736,...,Complete,"BCS,EC-KT,EC-HTOM,EC-AM,IRR,SNTC,SSPT,ST,PR,BRW",FIELD,NEW,NaN,NaT,False,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,BT,53506130,...,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,ST",FIELD,RENEW,NaN,NaT,False,,,
3358,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,BT,53506136,...,Complete,"BCS,BRW,DNA,EC-KT,IRR,ST",TARGETING,RENEW,NaN,NaT,True,,,
3359,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,BT,53506136,...,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,ST",FIELD,RENEW,NaN,NaT,False,,,
3360,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,BT,53506136,...,Complete,"BCS,BRW,DNA,EC-KT,IRR,ST",TARGETING,RENEW,NaN,NaT,True,,,


In [198]:
with pd.ExcelWriter(fin_path + "\\" + endfilename) as writer:  #endfilename  endfilename
    df.to_excel(writer, sheet_name = 'RawData', index=False)
    table.to_excel(writer, sheet_name = 'Pivot', index = False)
    compass_exists_table.to_excel(writer, sheet_name = "Compass Exists", index=False)